<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
#Keras provides the img_to_array() function for converting a loaded image in PIL format
#into a NumPy array for use with deep learning models
from tensorflow.keras.preprocessing.image import img_to_array
#array_to_img() function that can be used for converting a NumPy array of pixel data into a PIL
#Converts a class vector (integers) to binary class matrix.
from tensorflow.keras.utils import to_categorical
#To arrange the Keras layers in a sequential order
from tensorflow.keras.models import Sequential
#Flatten layer is used to make the multidimensional input one-dimensional,
#commonly used in the transition from the convolution layer to the full connected layer.
#defines the size of the output from the dense layer
from tensorflow.keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random

In [2]:
from google.colab import drive